In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import math

import string, unicodedata
#import contractions
#import inflect

#from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_columns', None) 

In [2]:
page=requests.get("http://www.dailymail.co.uk/sport/index.html")

In [3]:
soup=BeautifulSoup(page.content, 'html.parser')
links=[a['href'].split('#')[0] for a in soup.find_all('a',attrs={'href':re.compile("^http://www.dailymail.co.uk/sport")})]

In [4]:
len(links)

108

In [5]:
sportsarticles=[]

for link in links:
    body=requests.get(link)
    article=BeautifulSoup(body.content,'html.parser')
    paras=article.find_all('p',class_="mol-para-with-font")
    if paras:
        one=[]
        for text in paras:
            one.append(text.get_text())
    sportsarticles.append(one)

In [6]:
#removing nested list, converting each article as a big string instead
newlist=[]
for a in sportsarticles:
    b= ' '.join(a)
    newlist.append(b)

In [7]:
#removing duplicate articles
final_list = []
for num in newlist:
    if num not in final_list:
        final_list.append(num)

In [8]:
with open("articles8Aug.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in final_list:
        writer.writerow([val])

In [180]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rajat13440\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rajat13440\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\rajat13440\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\rajat13440\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\rajat13440\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rajat13440\AppData\Roaming\nltk_da

True

In [8]:
articles=pd.DataFrame({"articles":final_list})

In [39]:
articles=pd.read_csv("Backup.csv")

In [40]:
articles["Tokens"]=articles["articles"].apply(lambda x:nltk.word_tokenize(x))

In [41]:
articles["MsgSentenceTokens"]=articles["articles"].apply(lambda x:nltk.sent_tokenize(x))

In [15]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

In [16]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in nltk.corpus.stopwords.words('english'):
            new_words.append(word)
    return new_words

In [17]:
def lower(x):
    norm=[word.lower() for word in x]
    return norm

In [18]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [19]:
def normalize(x):
    x1=remove_stopwords(x)
    x2=remove_punctuation(x1)
    x3=lower(x2)
    #print(x3)
    #x3=remove_non_ascii(x2)
    return x3

In [42]:
articles.head()

,Tokens,MsgSentenceTokens,NormTokens,POS_Msg,Sentiments,Sentiments2,stems,lemmas,articles,TFIDF
0,"[No, wonder, Jose, Mourinho, is, worried, about, the, start, of, the, new, season, ., Four, days, to, go, until, it, kicks, off, at, Old, Trafford, and, he, is, still, waiting, for, the, real, Manchester, United, to, turn, up, ., This, has, been, a, strange, ,, disjointed, few, weeks, for, Mourinho, and, his, players, ,, and, it, showed, as, they, completed, their, pre-season, on, a, muggy, night, in, Munich, ., On, this, evidence, ,, whatever, team, United, put, out, on, Friday, night, will, struggle, to, overcome, Leicester, never, mind, challenge, Manchester, City, 's, dominance, of, the, Premier, League, ., BAYERN, MUNICH, ...]","[No wonder Jose Mourinho is worried about the start of the new season., Four days to go until it kicks off at Old Trafford and he is still waiting for the real Manchester United to turn up., This has been a strange, disjointed few weeks for Mourinho and his players, and it showed as they completed their pre-season on a muggy night in Munich., On this evidence, whatever team United put out on Friday night will struggle to overcome Leicester never mind challenge Manchester City's dominance of the Premier League., BAYERN MUNICH (3-4-3): Neuer; Kimmich (Rudy 72), Sule, Hummels (Boateng 46), Alaba (Bernat 56); Martinez; Robben, Muller, Thiago (Rafinha 60), Ribery; Gnabry., Subs: Ulreich, Hoffmann, Lewandowski, Boateng, Goretzka, Coman, Mai, Zylla, Franzke., Booked: Boateng Scorer: Martinez 59 MAN UTD (4-3-3): De Gea (Grant 78); Darmian (Jones 65), Bailly (Smalling 51), Lindelof (Tuanzebe 78), Shaw (Mitchell 78); Herrera (Garner 81), Pereira, Fred (Chong 78); Mata (McTominay 65), Rashfor...","['wonder', 'jose', 'mourinho', 'worried', 'start', 'new', 'season', 'four', 'days', 'go', 'kicks', 'old', 'trafford', 'still', 'waiting', 'real', 'manchester', 'united', 'turn', 'strange', 'disjointed', 'weeks', 'mourinho', 'players', 'showed', 'completed', 'preseason', 'muggy', 'night', 'munich', 'evidence', 'whatever', 'team', 'united', 'put', 'friday', 'night', 'struggle', 'overcome', 'leicester', 'never', 'mind', 'challenge', 'manchester', 'city', 'dominance', 'premier', 'league', 'bayern', 'munich', '343', 'neuer', 'kimmich', 'rudy', '72', 'sule', 'hummels', 'boateng', '46', 'alaba', 'bernat', '56', 'martinez', 'robben', 'muller', 'thiago', 'rafinha', '60', 'ribery', 'gnabry', 'subs', 'ulreich', 'hoffmann', 'lewandowski', 'boateng', 'goretzka', 'coman', 'mai', 'zylla', 'franzke', 'booked', 'boateng', 'scorer', 'martinez', '59', 'man', 'utd', '433', 'de', 'gea', 'grant', '78', 'darmian', 'jones', '65', 'bailly', 'smalling', '51', 'lindelof', 'tuanzebe', '78', 'shaw', 'mitchell'...","[('wonder', 'NN'), ('jose', 'NN'), ('mourinho', 'NN'), ('worried', 'VBD'), ('start', 'JJ'), ('new', 'JJ'), ('season', 'NN'), ('four', 'CD'), ('days', 'NNS'), ('go', 'VBP'), ('kicks', 'NNS'), ('old', 'JJ'), ('trafford', 'NN'), ('still', 'RB'), ('waiting', 'VBG'), ('real', 'JJ'), ('manchester', 'NN'), ('united', 'VBD'), ('turn', 'NN'), ('strange', 'JJ'), ('disjointed', 'VBD'), ('weeks', 'NNS'), ('mourinho', 'NN'), ('players', 'NNS'), ('showed', 'VBD'), ('completed', 'VBN'), ('preseason', 'NN'), ('muggy', 'NN'), ('night', 'NN'), ('munich', 'JJ'), ('evidence', 'NN'), ('whatever', 'WDT'), ('team', 'NN'), ('united', 'VBD'), ('put', 'NN'), ('friday', 'NN'), ('night', 'NN'), ('struggle', 'NN'), ('overcome', 'VBP'), ('leicester', 'NN'), ('never', 'RB'), ('mind', 'VBP'), ('challenge', 'JJ'), ('manchester', 'NN'), ('city', 'NN'), ('dominance', 'NN'), ('premier', 'JJR'), ('league', 'NN'), ('bayern', 'NN'), ('munich', '$'), ('343', 'CD'), ('neuer', 'NN'), ('kimmich', 'NN'), ('rudy', 'VBD'), ('7...",1,-1.0,"['wond', 'jos', 'mourinho', 'worry', 'start', 'new', 'season', 'four', 'day', 'go', 'kick', 'old', 'trafford', 'stil', 'wait', 'real', 'manchest', 'unit', 'turn', 'strange', 'disjoint', 'week', 'mourinho', 'play', 'show', 'complet', 'preseason', 

In [43]:
articles["NormTokens"]=articles["Tokens"].apply(lambda x: normalize(x))

In [44]:
articles["POS_Msg"]=articles["NormTokens"].apply(lambda x:nltk.pos_tag(x))

In [45]:
articles["POS_Msg2"]=articles["Tokens"].apply(lambda x:nltk.pos_tag(x))

In [38]:
#articles.drop("Tokens2",axis=1,inplace=True)

In [156]:
#articles["POS_Msg_Uni"]=articles["NormTokens"].apply(lambda x:nltk.pos_tag(x,tagset='universal'))

In [20]:
'''def noun_POS(x):
    words=nltk.pos_tag(x,tagset='universal')
    mw= [word[0] for word in words if word[1]=='NOUN' or word[1]=='VERB']
    return mw'''

In [21]:
#articles["POS_Msg_2"]=articles["NormTokens"].apply(lambda x:noun_POS(x))

In [157]:
def chunkMsg(x):
    entities=nltk.chunk.ne_chunk(x)
    return x

In [158]:
articles["Chunked"] = articles["POS_Msg"].apply(lambda x: chunkMsg(x))

In [162]:
len(articles["Chunked"][0])

403

In [163]:
len(articles["POS_Msg"][0])

403

In [25]:
articles.drop(["NormText"],axis=1,inplace=True)

In [46]:
articles.head()

,Tokens,MsgSentenceTokens,NormTokens,POS_Msg,Sentiments,Sentiments2,stems,lemmas,articles,TFIDF,POS_Msg2
0,"[No, wonder, Jose, Mourinho, is, worried, about, the, start, of, the, new, season, ., Four, days, to, go, until, it, kicks, off, at, Old, Trafford, and, he, is, still, waiting, for, the, real, Manchester, United, to, turn, up, ., This, has, been, a, strange, ,, disjointed, few, weeks, for, Mourinho, and, his, players, ,, and, it, showed, as, they, completed, their, pre-season, on, a, muggy, night, in, Munich, ., On, this, evidence, ,, whatever, team, United, put, out, on, Friday, night, will, struggle, to, overcome, Leicester, never, mind, challenge, Manchester, City, 's, dominance, of, the, Premier, League, ., BAYERN, MUNICH, ...]","[No wonder Jose Mourinho is worried about the start of the new season., Four days to go until it kicks off at Old Trafford and he is still waiting for the real Manchester United to turn up., This has been a strange, disjointed few weeks for Mourinho and his players, and it showed as they completed their pre-season on a muggy night in Munich., On this evidence, whatever team United put out on Friday night will struggle to overcome Leicester never mind challenge Manchester City's dominance of the Premier League., BAYERN MUNICH (3-4-3): Neuer; Kimmich (Rudy 72), Sule, Hummels (Boateng 46), Alaba (Bernat 56); Martinez; Robben, Muller, Thiago (Rafinha 60), Ribery; Gnabry., Subs: Ulreich, Hoffmann, Lewandowski, Boateng, Goretzka, Coman, Mai, Zylla, Franzke., Booked: Boateng Scorer: Martinez 59 MAN UTD (4-3-3): De Gea (Grant 78); Darmian (Jones 65), Bailly (Smalling 51), Lindelof (Tuanzebe 78), Shaw (Mitchell 78); Herrera (Garner 81), Pereira, Fred (Chong 78); Mata (McTominay 65), Rashfor...","[no, wonder, jose, mourinho, worried, start, new, season, four, days, go, kicks, old, trafford, still, waiting, real, manchester, united, turn, this, strange, disjointed, weeks, mourinho, players, showed, completed, preseason, muggy, night, munich, on, evidence, whatever, team, united, put, friday, night, struggle, overcome, leicester, never, mind, challenge, manchester, city, s, dominance, premier, league, bayern, munich, 343, neuer, kimmich, rudy, 72, sule, hummels, boateng, 46, alaba, bernat, 56, martinez, robben, muller, thiago, rafinha, 60, ribery, gnabry, subs, ulreich, hoffmann, lewandowski, boateng, goretzka, coman, mai, zylla, franzke, booked, boateng, scorer, martinez, 59, man, utd, 433, de, gea, grant, 78, darmian, jones, 65, bailly, ...]","[(no, DT), (wonder, NN), (jose, NN), (mourinho, NN), (worried, VBD), (start, JJ), (new, JJ), (season, NN), (four, CD), (days, NNS), (go, VBP), (kicks, NNS), (old, JJ), (trafford, NN), (still, RB), (waiting, VBG), (real, JJ), (manchester, NN), (united, VBD), (turn, NN), (this, DT), (strange, JJ), (disjointed, JJ), (weeks, NNS), (mourinho, NN), (players, NNS), (showed, VBD), (completed, VBN), (preseason, NN), (muggy, NN), (night, NN), (munich, NN), (on, IN), (evidence, NN), (whatever, NN), (team, NN), (united, VBD), (put, NN), (friday, NN), (night, NN), (struggle, NN), (overcome, VBP), (leicester, NN), (never, RB), (mind, VBP), (challenge, JJ), (manchester, NN), (city, NN), (s, NN), (dominance, NN), (premier, NN), (league, NN), (bayern, NN), (munich, $), (343, CD), (neuer, NN), (kimmich, NN), (rudy, VBD), (72, CD), (sule, NN), (hummels, NNS), (boateng, VBP), (46, CD), (alaba, JJ), (bernat, NN), (56, CD), (martinez, NN), (robben, NN), (muller, NN), (thiago, IN), (rafinha, NN), (60, CD...",1,-1.0,"['wond', 'jos', 'mourinho', 'worry', 'start', 'new', 'season', 'four', 'day', 'go', 'kick', 'old', 'trafford', 'stil', 'wait', 'real', 'manchest', 'unit', 'turn', 'strange', 'disjoint', 'week', 'mourinho', 'play', 'show', 'complet', 'preseason', 'muggy', 'night', 'munich', 'evid', 'whatev', 'team', 'unit', 'put', 'friday', 'night', 'struggle', 'overcom', 'leicest', 'nev', 'mind', 'challeng', 'manchest', 'city', 'domin', 'premy', 'leagu', 'bayern', 'munich', '343', 'neu', '

In [47]:
sentence=articles["POS_Msg"][0]

In [48]:
grammar = "NP: {<DT>?<JJ>*<NN>}"

In [49]:
cp = nltk.RegexpParser(grammar)

In [50]:
result = cp.parse(sentence)

In [51]:
print(result)

(S
  (NP no/DT wonder/NN)
  (NP jose/NN)
  (NP mourinho/NN)
  is/VBZ
  worried/VBN
  about/IN
  (NP the/DT start/NN)
  of/IN
  (NP the/DT new/JJ season/NN)
  ./.
  four/CD
  days/NNS
  to/TO
  go/VB
  until/IN
  it/PRP
  kicks/VBZ
  off/RP
  at/IN
  (NP old/JJ trafford/NN)
  and/CC
  he/PRP
  is/VBZ
  still/RB
  waiting/VBG
  for/IN
  (NP the/DT real/JJ manchester/NN)
  united/VBD
  to/TO
  turn/VB
  up/RP
  ./.
  this/DT
  has/VBZ
  been/VBN
  a/DT
  strange/JJ
  ,/,
  disjointed/VBD
  few/JJ
  weeks/NNS
  for/IN
  (NP mourinho/NN)
  and/CC
  his/PRP$
  players/NNS
  ,/,
  and/CC
  it/PRP
  showed/VBD
  as/IN
  they/PRP
  completed/VBD
  their/PRP$
  (NP pre-season/NN)
  on/IN
  (NP a/DT muggy/JJ night/NN)
  in/IN
  (NP munich/NN)
  ./.
  on/IN
  (NP this/DT evidence/NN)
  ,/,
  whatever/WDT
  (NP team/NN)
  united/VBD
  put/VBD
  out/RP
  on/IN
  (NP friday/JJ night/NN)
  will/MD
  struggle/VB
  to/TO
  overcome/VB
  (NP leicester/NN)
  never/RB
  mind/VBP
  challenge/JJ
  manchester

In [52]:
result.draw()

In [47]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [48]:
sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neg_word_list=[]
neu_word_list=[]

In [30]:
def GetSentiment(text):
    temp=[]
    ncount=0
    pcount=0
    for word in text:
    
        if (sid.polarity_scores(word)['compound']) >= 0.5:
            pcount+=1
            if word not in pos_word_list:
                pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.5:
            ncount+=1
            if word not in neg_word_list:
                neg_word_list.append(word)
        else:
            if word not in neu_word_list: 
                neu_word_list.append(word)
    if(pcount>0 and ncount==0):
        temp.append(1)
    elif(ncount%2>0):
        temp.append(-1)
    elif(ncount%2==0 and ncount==0):
        temp.append(1)
    else:
        temp.append(0)            
    sentiment="Pos: "+ str(pcount) +" Neg: "+ str(ncount)
    return temp[0]

In [49]:
articles["Sentiments"]=articles["NormTokens"].apply(lambda x:GetSentiment(x))

In [50]:
articles.head()

,Tokens,MsgSentenceTokens,NormTokens,POS_Msg,Sentiments,Sentiments2,stems,lemmas,articles,TFIDF,POS_Msg2
0,"[No, wonder, Jose, Mourinho, is, worried, about, the, start, of, the, new, season, ., Four, days, to, go, until, it, kicks, off, at, Old, Trafford, and, he, is, still, waiting, for, the, real, Manchester, United, to, turn, up, ., This, has, been, a, strange, ,, disjointed, few, weeks, for, Mourinho, and, his, players, ,, and, it, showed, as, they, completed, their, pre-season, on, a, muggy, night, in, Munich, ., On, this, evidence, ,, whatever, team, United, put, out, on, Friday, night, will, struggle, to, overcome, Leicester, never, mind, challenge, Manchester, City, 's, dominance, of, the, Premier, League, ., BAYERN, MUNICH, ...]","[No wonder Jose Mourinho is worried about the start of the new season., Four days to go until it kicks off at Old Trafford and he is still waiting for the real Manchester United to turn up., This has been a strange, disjointed few weeks for Mourinho and his players, and it showed as they completed their pre-season on a muggy night in Munich., On this evidence, whatever team United put out on Friday night will struggle to overcome Leicester never mind challenge Manchester City's dominance of the Premier League., BAYERN MUNICH (3-4-3): Neuer; Kimmich (Rudy 72), Sule, Hummels (Boateng 46), Alaba (Bernat 56); Martinez; Robben, Muller, Thiago (Rafinha 60), Ribery; Gnabry., Subs: Ulreich, Hoffmann, Lewandowski, Boateng, Goretzka, Coman, Mai, Zylla, Franzke., Booked: Boateng Scorer: Martinez 59 MAN UTD (4-3-3): De Gea (Grant 78); Darmian (Jones 65), Bailly (Smalling 51), Lindelof (Tuanzebe 78), Shaw (Mitchell 78); Herrera (Garner 81), Pereira, Fred (Chong 78); Mata (McTominay 65), Rashfor...","[no, wonder, jose, mourinho, worried, start, new, season, four, days, go, kicks, old, trafford, still, waiting, real, manchester, united, turn, this, strange, disjointed, weeks, mourinho, players, showed, completed, preseason, muggy, night, munich, on, evidence, whatever, team, united, put, friday, night, struggle, overcome, leicester, never, mind, challenge, manchester, city, s, dominance, premier, league, bayern, munich, 343, neuer, kimmich, rudy, 72, sule, hummels, boateng, 46, alaba, bernat, 56, martinez, robben, muller, thiago, rafinha, 60, ribery, gnabry, subs, ulreich, hoffmann, lewandowski, boateng, goretzka, coman, mai, zylla, franzke, booked, boateng, scorer, martinez, 59, man, utd, 433, de, gea, grant, 78, darmian, jones, 65, bailly, ...]","[(no, DT), (wonder, NN), (jose, NN), (mourinho, NN), (worried, VBD), (start, JJ), (new, JJ), (season, NN), (four, CD), (days, NNS), (go, VBP), (kicks, NNS), (old, JJ), (trafford, NN), (still, RB), (waiting, VBG), (real, JJ), (manchester, NN), (united, VBD), (turn, NN), (this, DT), (strange, JJ), (disjointed, JJ), (weeks, NNS), (mourinho, NN), (players, NNS), (showed, VBD), (completed, VBN), (preseason, NN), (muggy, NN), (night, NN), (munich, NN), (on, IN), (evidence, NN), (whatever, NN), (team, NN), (united, VBD), (put, NN), (friday, NN), (night, NN), (struggle, NN), (overcome, VBP), (leicester, NN), (never, RB), (mind, VBP), (challenge, JJ), (manchester, NN), (city, NN), (s, NN), (dominance, NN), (premier, NN), (league, NN), (bayern, NN), (munich, $), (343, CD), (neuer, NN), (kimmich, NN), (rudy, VBD), (72, CD), (sule, NN), (hummels, NNS), (boateng, VBP), (46, CD), (alaba, JJ), (bernat, NN), (56, CD), (martinez, NN), (robben, NN), (muller, NN), (thiago, IN), (rafinha, NN), (60, CD...",1,-1.0,"['wond', 'jos', 'mourinho', 'worry', 'start', 'new', 'season', 'four', 'day', 'go', 'kick', 'old', 'trafford', 'stil', 'wait', 'real', 'manchest', 'unit', 'turn', 'strange', 'disjoint', 'week', 'mourinho', 'play', 'show', 'complet', 'preseason', 'muggy', 'night', 'munich', 'evid', 'whatev', 'team', 'unit', 'put', 'friday', 'night', 'struggle', 'overcom', 'leicest', 'nev', 'mind', 'challeng', 'manchest', 'city', 'domin', 'premy', 'leagu', 'bayern', 'munich', '343', 'neu', '

In [33]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

In [34]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [35]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

In [36]:
articles['stems'] = articles["NormTokens"].apply(lambda x: stem_words(x))

In [37]:
articles['lemmas'] = articles["NormTokens"].apply(lambda x: lemmatize_verbs(x))

In [51]:
articles.to_csv("processedData6Aug.csv",sep=',',encoding='utf-8',index=False)

In [52]:
type(articles["NormTokens"][0])

list

In [77]:
def tf2(data,filename):
    fd=nltk.FreqDist(data[filename])
    return fd

In [74]:
def idf2(data,term):
    count=[term in row for row in data]
    inv_feq=math.log(len(count)/sum(count))
    return inv_feq

In [79]:
def tfidf2(data,row,n):
    term_scores={}
    file_fd=tf2(data,row)
    for term in file_fd:
        print(term)
        if term.isalpha():
            idf_val=idf2(data,term)
            tf_val=tf2(data,row)[term]
            tfidf_val=tf_val*idf_val
            term_scores[term]=round(tfidf_val,2)
    return sorted(term_scores.items(),key=lambda x:-x[1])[:n]

In [78]:
mylist=[]
for index, row in articles.iterrows():
    mylist.append(tfidf2(articles["NormTokens"],index,5))

In [80]:
mylist

[[('boateng', 14.28),
  ('friday', 13.61),
  ('martinez', 11.32),
  ('thiago', 11.32),
  ('mourinho', 10.96)],
 [('click', 5.71),
  ('mina', 5.04),
  ('leaves', 4.47),
  ('resolved', 4.47),
  ('app', 4.47)],
 [('major', 86.53),
  ('fare', 75.46),
  ('ins', 75.46),
  ('outs', 75.46),
  ('they', 62.35)],
 [('boateng', 14.28),
  ('united', 7.52),
  ('bayern', 6.81),
  ('woodward', 6.16),
  ('chief', 5.71)],
 [('mina', 10.08),
  ('barcelona', 7.92),
  ('virginia', 7.55),
  ('colombia', 6.16),
  ('digne', 6.16)],
 [('jesus', 8.02),
  ('teams', 5.94),
  ('frontrunners', 4.47),
  ('secondconsecutive', 4.47),
  ('pile', 4.47)],
 [('rashford', 24.64),
  ('jersey', 11.32),
  ('shirt', 9.07),
  ('wearing', 8.02),
  ('mourinho', 7.31)],
 [('caballero', 18.86),
  ('aguero', 17.83),
  ('foden', 15.12),
  ('rudiger', 11.32),
  ('morata', 10.1)],
 [('appearance', 7.16),
  ('city', 7.06),
  ('wembley', 7.03),
  ('prior', 6.73),
  ('gundogan', 6.73)],
 [('foden', 37.8),
  ('southgate', 12.6),
  ('unforg